In [1]:
import os
from dotenv import load_dotenv
#
#from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

sep = os.sep

In [2]:
load_dotenv()

hf_token = os.getenv('HF_TOKEN')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

os.environ['HF_TOKEN'] = hf_token
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token
# LangSmith tracking
#os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
#
os.environ['GROQ_API_KEY'] = groq_api_key

### Заменить OpenAIEmbeddings

In [3]:
'''
from langchain_community.embeddings import HuggingFaceEmbeddings

model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#model_kwargs = {'device': 'cuda'}
model_kwargs = {'device': 'cpu'}

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name=model_id,
    model_kwargs=model_kwargs
)
'''
pass

In [3]:
#model_id = 'sentence-transformers/all-MiniLM-l6-v2'
model_id = 'BAAI/bge-m3'

#model_args = {'device': 'cuda'}
model_args = {'device': 'cpu'}

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_id,
    model_kwargs=model_args,
    encode_kwargs={'normalize_embeddings': True}
)

e:\Progi\PythonProg\NLP\RAG\SunnySavita\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loder = TextLoader(f'..{sep}data{sep}speech.txt', encoding='utf-8')
document = loder.load()
len(document), type(document)

(1, list)

In [5]:
#print(document[0].page_content)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_documents(document)
len(text_chunks), type(text_chunks)

(99, list)

In [7]:
#print(text_chunks[0].page_content); print(end='\n\n\n')
print(text_chunks[98].page_content)

Сейчас наш момент встретить и преодолеть вызовы нашего времени.

И мы сделаем это, как один народ.

Одна Америка.

Соединенные Штаты Америки.

Да благословит вас всех Бог. Да защитит Бог наши войска.


In [8]:
vectorstore = FAISS.from_documents(text_chunks, huggingface_embeddings)

In [9]:
promt_template = '''
Ты - Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.
Используй следующие фрагменты релевантного контекста, чтобы ответить на вопрос.
Если ты не знаешь ответ, просто скажи, что не знаю.
Используй максимум двадцать предложений и сделай ответ кратким.
Question: {question}
Context: {context}
Answer:
'''

In [10]:
prompt = ChatPromptTemplate.from_template(promt_template)
retriever = vectorstore.as_retriever()
output_parser = StrOutputParser()

#llm_model = Ollama(model='sutyrin/saiga_mistral_7b', base_url='http://localhost:11434')
llm_model = OllamaLLM(model='sutyrin/saiga_mistral_7b', base_url='http://localhost:11434')

In [11]:
rag_chain = (
    {'context': retriever,  'question': RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [12]:
rag_chain.invoke('Какую поддержку США оказывают Украине в экономическом и военном отношении?')

'США оказывают поддержку Украине в экономическом и военном отношении. Военная помощь включает прямую финансовую поддержку, а также предоставление вооружения и оборудования. Экономическая помощь заключается в выделении более 1 миллиарда долларов США в качестве прямой помощи Украине. Также США оказывают гуманитарную поддержку, например, помогая украинскому народу облегчить его страдания.'

In [14]:
rag_chain.invoke('Какие действия предпринимают США для решения проблемы роста цен на газ?')

'США предпринимают следующие действия для решения проблемы роста цен на газ:\n1. Сотрудничество с 30 другими странами для высвобождения 60 миллионов баррелей нефти из резервов по всему миру.\n2. Высвобождение 30 миллионов баррелей из национального Стратегического нефтяного резерва США.\n3. Развитие инфраструктуры и инноваций в Америке для производства большего количества автомобилей и полупроводников.'